In [2]:
import sys, os
from pathlib import Path

sys.path.append(os.path.abspath(Path().resolve().parent/'src'))

In [3]:
from kinematicsrobotics.datahandler import Extract
from kinematicsrobotics.dataprocessing import Preprocessing
from kinematicsrobotics.model import Model
from kinematicsrobotics.metrics import Metrics
from kinematicsrobotics.kinematics import Robo

## Dataset

In [4]:
ext = Extract()
ext._path_project = os.path.abspath(Path().resolve().parent)

dataset = ext.dataframe( r'src\data\ready\dataset-radius-1.5cm.csv')

In [5]:
data  = Preprocessing(dataset = dataset, 
                           x_labels=['p_x', 'p_y','p_z', 'roll', 'pich', 'yaw'],
                           y_labels=['theta_1', 'theta_2', 'theta_3', 'theta_4']
                        )

In [6]:
x_train, x_test, y_train, y_test = data.data_train_test

## Modelo

In [7]:
mlp = Model.mlp_regressor(hidden_layer_sizes=(100,100,100,100,100), EPOCHS_NOCHANGE = 20)

## Estimação de Parâmatros

In [8]:
# Estimação de hiperparâmetros
history = ext.dataframe(r'src\data\ready\history.csv')

history_best = history[history['rank_test_score'] == 1]

params = eval(history_best.iloc[0]['params'])

## Treino

In [9]:
mlp.set_model(**params)

In [10]:
mlp.fit(x=x_train,y=y_train)

## Metricas

In [11]:
metrica = Metrics(model = mlp, 
                  preprocessing = data)

### Erro juntas

In [12]:
metrica.predict_joint(x = x_train)

array([[ 65.63728466,  37.64542696,  84.25842577,  46.86745838],
       [ 96.175419  ,  95.18336342,  83.34731067,  60.30910712],
       [ 35.55518928,  83.57556657,  83.73180339,  36.2507363 ],
       ...,
       [ 93.95269506, 119.35308256,  10.48478097,  35.50424211],
       [ 19.7962104 ,  36.55654538,  85.09270133,  35.67230024],
       [ 35.31630349,  36.16963876,  47.16363729,  -0.93332684]])

In [13]:
metrica.mse_joint(x = x_train, y = y_train)

array([0.55269527, 1.56803829, 5.15688068, 2.22570988])

### Erro cartesiano

In [14]:
Elos = [['theta_1',10,0,90,0],
        ['theta_2',0,18,180,0],
        ['theta_3',0,18,-180,0],
        ['theta_4',0,0,90,90],
        ['theta_5',18,0,0,0]
]

robo = Robo("Robo",Elos)

metrica.robotic(robo = robo)

In [15]:
metrica.predict_operacional(x = x_train)

In [ ]:
metrica.mse_operacional(x = x_train, y = y_train)

array([[ 40.,  48.,  60.,   0.],
       [  5.,  48.,  72.,   0.],
       [ 85.,  36.,  72.,  48.],
       ...,
       [ 70., 108.,  12.,  48.],
       [ 75.,  36.,  36.,   0.],
       [ 10.,  48.,  96.,  36.]])